<a href="https://colab.research.google.com/github/styang3first/DeepLearningNotes/blob/main/TensorFlow_RLwithDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
from collections import deque
import gym

from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
env.reset()
for step in range(1000):
  env.render(mode='human')
  random_action = env.action_space.sample()
  env.step(random_action)
env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/envs/classic_control/cartpole.py:179: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [ ]:
# 4 observations
num_observations = env.observation_space.shape[0]
num_actions = env.action_space.n
num_observations, num_actions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(4, 2)

In [ ]:
# input_shape = 4 ANN ---> neurons==actions
model = Sequential()
model.add(Dense(16, input_shape=(1,num_observations)))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
# Neurons == action_space
model.add(Dense(num_actions))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer=(Adam(lr=LEARNING_RATE)))
print(model.summary())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 1, 16)             80        
                                                                 
 activation_15 (Activation)  (None, 1, 16)             0         
                                                                 
 dense_28 (Dense)            (None, 1, 32)             544       
                                                                 
 activation_16 (Activation)  (None, 1, 32)             0         
                                                                 
 dense_29 (Dense)            (None, 1, 2)              66        
                                                                 
 activation_17 (Activation)  (None, 1, 2)              0         
                                                                 
Total params: 690 (2.70 KB)
Trainable params: 690 (2.

same as:
```python
# input_shape = 4 ANN ---> neurons==actions
model = Sequential()
model.add(Dense(16, input_shape=(1,num_observations), activation='relu'))
model.add(Dense(32, activation='relu'))
# Neurons == action_space
model.add(Dense(num_actions, activation='linear'))
print(model.summary())
```

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def epsilon_greedy_action_selection(model, epsilon,observation):
  if np.random.random()>epsilon:
    prediction = model.prediction(observation)
    action = np.argmax(prediction)
  else:
    action = np.random.randint(0,env.action_space.n)
  return action
def replay(reply_buffer, batch_size, model, target_model):
  if len(replay_buffer)<batch_size:
    return
  samples = random.sample(reply_buffer,batch_size)

  target_batch = []
  zipped_samples = list(zip(*samples)) # extract the same idex of each samples
  states, actions, rewards, new_states, dones = zipped_samples # exp(s_t, a_t, r_t+1, s_t+1)
  targets = target_model.predict(np.array(states))
  q_values = model.predict(np.array(new_states))

  for i in range(batch_size):
    q_value = max(q_values[i][0])
    target = targets[i].copy()
    if dones[i]:
      target[0][actions[i]] = rewards[i]
    else:
      target[0][actions[i]] = rewards[i] + GAMMA*q_value
    target_batch.append(target)
  model.fit(np.array(states), np.array(target_batch),epochs=1,verbose=0)

def update_model_handler(epoch,update_target_model,model,target_model):
  if epoch>0 and epoch % update_target_model==0:
    target_model.set_weights(model.get_weights()) # periodically update Q_target

target_model=clone_model(model)

EPOCHS = 1000
epsilon = 1.0

EPSILON_REDUCE = 1-0.005
LEARNING_RATE = 0.001


GAMMA = 0.95

BATCH_SIZE = 32
replay_buffer = deque(maxlen=20000)
update_target_model = 10




In [ ]:
def epsilon_greedy_action_selection(model, epsilon, observation):
    if np.random.random() > epsilon:
        prediction = model.predict(observation)  # perform the prediction on the observation
        action = np.argmax(prediction)  # Chose the action with the higher value
    else:
        action = np.random.randint(0, env.action_space.n)  # Else use random action
    return action
def replay(replay_buffer, batch_size, model, target_model):

    # As long as the buffer has not enough elements we do nothing
    if len(replay_buffer) < batch_size:
        return

    # Take a random sample from the buffer with size batch_size
    samples = random.sample(replay_buffer, batch_size)

    # to store the targets predicted by the target network for training
    target_batch = []

    # Efficient way to handle the sample by using the zip functionality
    zipped_samples = list(zip(*samples))
    states, actions, rewards, new_states, dones = zipped_samples

    # Predict targets for all states from the sample
    targets = target_model.predict(np.array(states))

    # Predict Q-Values for all new states from the sample
    q_values = model.predict(np.array(new_states))

    # Now we loop over all predicted values to compute the actual targets
    for i in range(batch_size):

        # Take the maximum Q-Value for each sample
        q_value = max(q_values[i][0])

        # Store the ith target in order to update it according to the formula
        target = targets[i].copy()
        if dones[i]:
            target[0][actions[i]] = rewards[i]
        else:
            target[0][actions[i]] = rewards[i] + q_value * GAMMA
        target_batch.append(target)

    # Fit the model based on the states and the updated targets for 1 epoch
    model.fit(np.array(states), np.array(target_batch), epochs=1, verbose=0)
def update_model_handler(epoch, update_target_model, model, target_model):
    if epoch > 0 and epoch % update_target_model == 0:
        target_model.set_weights(model.get_weights())



In [ ]:
best_so_far=0
for epoch in range(EPOCHS):
  observation = env.reset()
  # (1,X) [a,b,c,d] .reshape --> (1,4)
  observation = observation.reshape([1,4])
  done = False

  points = 0
  while not done:
    action = epsilon_greedy_action_selection(model,epsilon,observation)
    next_observation,reward,done,info = env.step(action)
    next_observation = next_observation.reshape([1,4])
    replay_buffer.append((observation,action,reward,next_observation,done)) # zipped_sample takes value from here
    observation = next_observation

    points += 1
    replay(replay_buffer,BATCH_SIZE,model,target_model)
  epsilon *= EPSILON_REDUCE
  update_model_handler(epoch,update_target_model,model,target_model)

  if points > best_so_far:
    points = best_so_far
  if epoch%25 == 0:
    print(f"{epoch}: POINTS: {points} eps: {epsilon} BSF: {best_so_far}")


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 8s 8s/step


In [ ]:
GAMMA

NameError: ignored